<a href="https://colab.research.google.com/github/janaghoniem/Social-Media-Sentiment-Analysis/blob/main/ML_Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import vstack
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import pandas as pd
import numpy as np
import ast
import re
import pickle
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import drive

### Arabic Testing

In [2]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data Science/Arabic Sentiment/Datasets/arabic_sentiment_reviews.csv')
with open("/content/drive/MyDrive/Colab Notebooks/Data Science/Arabic Sentiment/Features/training_data.pkl", 'rb') as f:
    training_data = pickle.load(f)

Mounted at /content/drive


### 1) Random forest

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder, label_binarize
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

# Split data into train, validation, test
X_temp, X_test, y_temp, y_test = train_test_split(X, y_cat, test_size=0.2, stratify=y_cat, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# Prepare labels for sklearn (not one-hot)
y_train_sklearn = np.argmax(y_train, axis=1)
y_val_sklearn = np.argmax(y_val, axis=1)
y_test_sklearn = np.argmax(y_test, axis=1)

# Train initial Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42)
rf_model.fit(X_train, y_train_sklearn)

# Evaluate initial model
print("Initial Random Forest Training Accuracy:", accuracy_score(y_train_sklearn, rf_model.predict(X_train)))
print("Initial Random Forest Validation Accuracy:", accuracy_score(y_val_sklearn, rf_model.predict(X_val)))
print("Initial Random Forest Testing Accuracy:", accuracy_score(y_test_sklearn, rf_model.predict(X_test)))

print("--------------------------------------------------")

# Combine train and validation for tuning
X_train_val = np.vstack([X_train, X_val])
y_train_val = np.concatenate([y_train_sklearn, y_val_sklearn])

# Hyperparameter tuning with GridSearchCV
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [20, 30, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_grid = GridSearchCV(RandomForestClassifier(random_state=42),
                       rf_params,
                       cv=3,
                       n_jobs=-1,
                       verbose=1)
rf_grid.fit(X_train_val, y_train_val)

# Best model after tuning
best_rf = rf_grid.best_estimator_

# Evaluate best model
print("Best Parameters:", rf_grid.best_params_)
print("Best Random Forest Training Accuracy:", accuracy_score(y_train_sklearn, best_rf.predict(X_train)))
print("Best Random Forest Validation Accuracy:", accuracy_score(y_val_sklearn, best_rf.predict(X_val)))
print("Best Random Forest Testing Accuracy:", accuracy_score(y_test_sklearn, best_rf.predict(X_test)))

print("--------------------------------------------------")

# Evaluate best model with additional metrics on the test set
y_test_pred = best_rf.predict(X_test)
y_test_proba = best_rf.predict_proba(X_test)

# Accuracy
print("Test Accuracy:", accuracy_score(y_test_sklearn, y_test_pred))

# Precision, Recall, F1-score (macro because it's multiclass)
print("Test Precision (Macro):", precision_score(y_test_sklearn, y_test_pred, average='macro'))
print("Test Recall (Macro):", recall_score(y_test_sklearn, y_test_pred, average='macro'))
print("Test F1-Score (Macro):", f1_score(y_test_sklearn, y_test_pred, average='macro'))

# Classification report
print("\nClassification Report:\n", classification_report(y_test_sklearn, y_test_pred, target_names=label_encoder.classes_))

# AUC-ROC (requires one-hot encoded y_test)
y_test_bin = label_binarize(y_test_sklearn, classes=np.arange(len(label_encoder.classes_)))

# Handle multiclass AUC
try:
    auc_roc = roc_auc_score(y_test_bin, y_test_proba, multi_class='ovr', average='macro')
    print("Test AUC-ROC (Macro OVR):", auc_roc)
except ValueError as e:
    print("AUC-ROC Error:", e)


KeyboardInterrupt: 

### 2) SVM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']  # Use your dense FastText embeddings
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

# Split into train, validation, test (stratifying on labels)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# ----------------- Baseline SVM -----------------
print("\nTraining Baseline SVM...")
svm_baseline = LinearSVC(max_iter=10000, random_state=42, dual=False)
svm_baseline.fit(X_train, y_train)

# Predictions
train_preds = svm_baseline.predict(X_train)
val_preds = svm_baseline.predict(X_val)
test_preds = svm_baseline.predict(X_test)

print("\nBaseline SVM Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# ----------------- SVM Hyperparameter Tuning -----------------
print("\nStarting SVM hyperparameter tuning...")
svm_params = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],
    'loss': ['squared_hinge'],
    'dual': [False]
}

svm_grid = GridSearchCV(
    LinearSVC(random_state=42, max_iter=10000),
    svm_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit ONLY on X_train (avoid stacking with val)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_

# Predictions after tuning
svm_train_preds = best_svm.predict(X_train)
svm_val_preds = best_svm.predict(X_val)
svm_test_preds = best_svm.predict(X_test)

print("\nTuned SVM Results:")
print("Best Parameters:", svm_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, svm_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, svm_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, svm_test_preds))



Training Baseline SVM...

Baseline SVM Results:
Training Accuracy: 0.798956228956229
Validation Accuracy: 0.7990909090909091
Testing Accuracy: 0.7961818181818182

Starting SVM hyperparameter tuning...
Fitting 3 folds for each of 4 candidates, totalling 12 fits

Tuned SVM Results:
Best Parameters: {'C': 10, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l2'}
Training Accuracy: 0.7989814814814815
Validation Accuracy: 0.7992803030303031
Testing Accuracy: 0.7964545454545454


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report ----
print("\nClassification Report (Test Data):")
print(classification_report(y_test, test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function) ----
try:
    test_decision_scores = svm_baseline.decision_function(X_test)
    auc = roc_auc_score(y_test, test_decision_scores, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR):", auc)
except Exception as e:
    print("AUC-ROC could not be calculated:", e)

from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report (Tuned SVM) ----
print("\nClassification Report (Tuned SVM - Test Data):")
print(classification_report(y_test, svm_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function for Tuned SVM) ----
try:
    test_decision_scores_tuned = best_svm.decision_function(X_test)
    auc_tuned = roc_auc_score(y_test, test_decision_scores_tuned, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned SVM:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned SVM) could not be calculated:", e)




Classification Report (Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (macro average OvR): 0.8760152659478608

Classification Report (Tuned SVM - Test Data):
              precision    recall  f1-score   support

           0       0.79      0.81      0.80     32629
           1       0.81      0.79      0.80     33371

    accuracy                           0.80     66000
   macro avg       0.80      0.80      0.80     66000
weighted avg       0.80      0.80      0.80     66000

AUC-ROC (macro average OvR) Tuned SVM: 0.8760306967319824


### 3) Logistic regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']  # Use your dense FastText embeddings
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y_cat = tf.keras.utils.to_categorical(y)

# Split into train, validation, test (stratifying on labels)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# ----------------- Baseline Logistic Regression -----------------
print("\nTraining Baseline Logistic Regression...")
logreg_baseline = LogisticRegression(max_iter=10000, random_state=42, solver='lbfgs', multi_class='multinomial')
logreg_baseline.fit(X_train, y_train)

# Predictions
train_preds = logreg_baseline.predict(X_train)
val_preds = logreg_baseline.predict(X_val)
test_preds = logreg_baseline.predict(X_test)

print("\nBaseline Logistic Regression Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# ----------------- Logistic Regression Hyperparameter Tuning -----------------
print("\nStarting Logistic Regression hyperparameter tuning...")
logreg_params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs'],  # Good for multiclass
    'multi_class': ['multinomial']
}

logreg_grid = GridSearchCV(
    LogisticRegression(max_iter=10000, random_state=42),
    logreg_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit ONLY on X_train
logreg_grid.fit(X_train, y_train)
best_logreg = logreg_grid.best_estimator_

# Predictions after tuning
logreg_train_preds = best_logreg.predict(X_train)
logreg_val_preds = best_logreg.predict(X_val)
logreg_test_preds = best_logreg.predict(X_test)

print("\nTuned Logistic Regression Results:")
print("Best Parameters:", logreg_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, logreg_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, logreg_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, logreg_test_preds))



Training Baseline Logistic Regression...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(



Baseline Logistic Regression Results:
Training Accuracy: 0.796986531986532
Validation Accuracy: 0.7985227272727272
Testing Accuracy: 0.7941666666666667

Starting Logistic Regression hyperparameter tuning...
Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, binary problems will be fit as proper binary  logistic regression models (as if multi_class='ovr' were set). Leave it to its default value to avoid this warning.
  warnings.warn(



Tuned Logistic Regression Results:
Best Parameters: {'C': 100, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'lbfgs'}
Training Accuracy: 0.7981060606060606
Validation Accuracy: 0.798939393939394
Testing Accuracy: 0.7962575757575757


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report (Baseline Logistic Regression) ----
print("\nClassification Report (Baseline Logistic Regression - Test Data):")
print(classification_report(y_test, test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function for Baseline Logistic Regression) ----
try:
    test_decision_scores_baseline = logreg_baseline.decision_function(X_test)
    auc_baseline = roc_auc_score(y_test, test_decision_scores_baseline, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Baseline Logistic Regression:", auc_baseline)
except Exception as e:
    print("AUC-ROC (Baseline Logistic Regression) could not be calculated:", e)


# ---- Classification Report (Tuned Logistic Regression) ----
print("\nClassification Report (Tuned Logistic Regression - Test Data):")
print(classification_report(y_test, logreg_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (using decision_function for Tuned Logistic Regression) ----
try:
    test_decision_scores_tuned = best_logreg.decision_function(X_test)
    auc_tuned = roc_auc_score(y_test, test_decision_scores_tuned, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned Logistic Regression:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned Logistic Regression) could not be calculated:", e)



Classification Report (Baseline Logistic Regression - Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (macro average OvR) Baseline Logistic Regression: 0.8741980417747179

Classification Report (Tuned Logistic Regression - Test Data):
              precision    recall  f1-score   support

           0       0.79      0.80      0.80     32629
           1       0.81      0.79      0.80     33371

    accuracy                           0.80     66000
   macro avg       0.80      0.80      0.80     66000
weighted avg       0.80      0.80      0.80     66000

AUC-ROC (macro average OvR) Tuned Logistic Regression: 0.875517256263241


### 4) Naive Bayes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load features and labels
X = training_data['X_fasttext']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

# Split into train, validation, test (stratifying on labels)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# ----------------- Baseline Gaussian Naive Bayes -----------------
print("\nTraining Baseline Gaussian Naive Bayes...")
nb_baseline = GaussianNB()
nb_baseline.fit(X_train, y_train)

# Predictions
train_preds = nb_baseline.predict(X_train)
val_preds = nb_baseline.predict(X_val)
test_preds = nb_baseline.predict(X_test)

print("\nBaseline Gaussian Naive Bayes Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# ----------------- Naive Bayes Hyperparameter Tuning -----------------
# For GaussianNB, the main hyperparameter is var_smoothing (default: 1e-9)
print("\nStarting Gaussian Naive Bayes hyperparameter tuning...")
nb_params = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

nb_grid = GridSearchCV(
    GaussianNB(),
    nb_params,
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Fit ONLY on X_train
nb_grid.fit(X_train, y_train)
best_nb = nb_grid.best_estimator_

# Predictions after tuning
nb_train_preds = best_nb.predict(X_train)
nb_val_preds = best_nb.predict(X_val)
nb_test_preds = best_nb.predict(X_test)

print("\nTuned Gaussian Naive Bayes Results:")
print("Best Parameters:", nb_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, nb_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, nb_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, nb_test_preds))



Training Baseline Gaussian Naive Bayes...

Baseline Gaussian Naive Bayes Results:
Training Accuracy: 0.6434175084175084
Validation Accuracy: 0.6449621212121213
Testing Accuracy: 0.6429090909090909

Starting Gaussian Naive Bayes hyperparameter tuning...
Fitting 3 folds for each of 5 candidates, totalling 15 fits

Tuned Gaussian Naive Bayes Results:
Best Parameters: {'var_smoothing': 1e-05}
Training Accuracy: 0.6433964646464646
Validation Accuracy: 0.645
Testing Accuracy: 0.6428939393939394


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# ---- Classification Report (Baseline Naive Bayes) ----
print("\nClassification Report (Baseline Gaussian Naive Bayes - Test Data):")
print(classification_report(y_test, test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (Baseline Naive Bayes) ----
try:
    test_prob_scores_baseline = nb_baseline.predict_proba(X_test)
    auc_baseline = roc_auc_score(y_test, test_prob_scores_baseline, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Baseline Naive Bayes:", auc_baseline)
except Exception as e:
    print("AUC-ROC (Baseline Naive Bayes) could not be calculated:", e)

# ---- Classification Report (Tuned Naive Bayes) ----
print("\nClassification Report (Tuned Gaussian Naive Bayes - Test Data):")
print(classification_report(y_test, nb_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# ---- AUC-ROC (Tuned Naive Bayes) ----
try:
    test_prob_scores_tuned = best_nb.predict_proba(X_test)
    auc_tuned = roc_auc_score(y_test, test_prob_scores_tuned, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned Naive Bayes:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned Naive Bayes) could not be calculated:", e)




Classification Report (Baseline Gaussian Naive Bayes - Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (Baseline Naive Bayes) could not be calculated: y should be a 1d array, got an array of shape (66000, 2) instead.

Classification Report (Tuned Gaussian Naive Bayes - Test Data):
              precision    recall  f1-score   support

           0       0.61      0.77      0.68     32629
           1       0.70      0.52      0.60     33371

    accuracy                           0.64     66000
   macro avg       0.65      0.64      0.64     66000
weighted avg       0.65      0.64      0.64     66000

AUC-ROC (Tuned Naive Bayes) could not be calculated: y should be a 1d array, got an array o

### 5) MLP

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# -------------------- Load Features and Labels --------------------
X = training_data['X_fasttext']  # FastText embeddings as dense vectors
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

# One-hot encoded labels (for Keras models)
y_cat = tf.keras.utils.to_categorical(y)

# -------------------- Train/Validation/Test Split --------------------
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# -------------------- Define MLP Model --------------------
def create_mlp(hidden_units=128, dropout_rate=0.5, learning_rate=0.001):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X.shape[1],)),
        tf.keras.layers.Dense(hidden_units, activation='relu'),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# -------------------- Baseline MLP Training --------------------
print("\nTraining Baseline MLP...")
baseline_mlp = create_mlp()
baseline_mlp.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data=(X_val, y_val))

# Predictions
train_preds = np.argmax(baseline_mlp.predict(X_train), axis=1)
val_preds = np.argmax(baseline_mlp.predict(X_val), axis=1)
test_preds = np.argmax(baseline_mlp.predict(X_test), axis=1)

print("\nBaseline MLP Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# -------------------- MLP Hyperparameter Tuning --------------------
print("\nStarting MLP hyperparameter tuning...")

keras_model = KerasClassifier(build_fn=create_mlp, verbose=0)

mlp_params = {
    'hidden_units': [64, 128],
    'dropout_rate': [0.3, 0.5],
    'learning_rate': [0.001, 0.0005],
    'batch_size': [32],
    'epochs': [15]
}

mlp_grid = GridSearchCV(estimator=keras_model,
                        param_grid=mlp_params,
                        cv=3,
                        n_jobs=-1,
                        verbose=1)

mlp_grid.fit(X_train, y_train)
best_mlp = mlp_grid.best_estimator_

# Predictions after tuning
mlp_train_preds = np.argmax(best_mlp.model.predict(X_train), axis=1)
mlp_val_preds = np.argmax(best_mlp.model.predict(X_val), axis=1)
mlp_test_preds = np.argmax(best_mlp.model.predict(X_test), axis=1)

print("\nTuned MLP Results:")
print("Best Parameters:", mlp_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, mlp_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, mlp_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, mlp_test_preds))


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# -------------------- Baseline MLP Classification Report and AUC-ROC --------------------
print("\nClassification Report (Baseline MLP - Test Data):")
print(classification_report(y_test, test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# AUC-ROC for Baseline MLP
try:
    test_probs_baseline = baseline_mlp.predict(X_test)  # shape: (n_samples, n_classes)
    auc_baseline = roc_auc_score(tf.keras.utils.to_categorical(y_test), test_probs_baseline, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Baseline MLP:", auc_baseline)
except Exception as e:
    print("AUC-ROC (Baseline MLP) could not be calculated:", e)

# -------------------- Tuned MLP Classification Report and AUC-ROC --------------------
print("\nClassification Report (Tuned MLP - Test Data):")
print(classification_report(y_test, mlp_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

# AUC-ROC for Tuned MLP
try:
    test_probs_tuned = best_mlp.model.predict(X_test)  # shape: (n_samples, n_classes)
    auc_tuned = roc_auc_score(tf.keras.utils.to_categorical(y_test), test_probs_tuned, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned MLP:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned MLP) could not be calculated:", e)


### 6) XGBoost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

# -------------------- Load Features and Labels --------------------
X = training_data['X_fasttext']  # FastText dense vectors
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

# -------------------- Train/Validation/Test Split --------------------
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1, stratify=y_temp, random_state=42)

# -------------------- Baseline XGBoost Training --------------------
print("\nTraining Baseline XGBoost...")

baseline_xgb = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(np.unique(y)),
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42
)

baseline_xgb.fit(X_train, y_train)

# Predictions
train_preds = baseline_xgb.predict(X_train)
val_preds = baseline_xgb.predict(X_val)
test_preds = baseline_xgb.predict(X_test)

print("\nBaseline XGBoost Results:")
print("Training Accuracy:", accuracy_score(y_train, train_preds))
print("Validation Accuracy:", accuracy_score(y_val, val_preds))
print("Testing Accuracy:", accuracy_score(y_test, test_preds))

# -------------------- XGBoost Hyperparameter Tuning --------------------
print("\nStarting XGBoost hyperparameter tuning...")

xgb_params = {
    'max_depth': [3, 6],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200],
    'subsample': [0.8],
    'colsample_bytree': [0.8]
}

xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(np.unique(y)),
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42
)

xgb_grid = GridSearchCV(estimator=xgb_model,
                        param_grid=xgb_params,
                        cv=3,
                        n_jobs=-1,
                        verbose=1)

xgb_grid.fit(X_train, y_train)
best_xgb = xgb_grid.best_estimator_

# Predictions after tuning
xgb_train_preds = best_xgb.predict(X_train)
xgb_val_preds = best_xgb.predict(X_val)
xgb_test_preds = best_xgb.predict(X_test)

print("\nTuned XGBoost Results:")
print("Best Parameters:", xgb_grid.best_params_)
print("Training Accuracy:", accuracy_score(y_train, xgb_train_preds))
print("Validation Accuracy:", accuracy_score(y_val, xgb_val_preds))
print("Testing Accuracy:", accuracy_score(y_test, xgb_test_preds))


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# -------------------- Baseline XGBoost Classification Report and AUC-ROC --------------------
print("\nClassification Report (Baseline XGBoost - Test Data):")
baseline_preds = xgb_model.predict(X_test)
print(classification_report(y_test, baseline_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

try:
    baseline_probs = xgb_model.predict_proba(X_test)  # shape: (n_samples, n_classes)
    auc_baseline = roc_auc_score(tf.keras.utils.to_categorical(y_test), baseline_probs, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Baseline XGBoost:", auc_baseline)
except Exception as e:
    print("AUC-ROC (Baseline XGBoost) could not be calculated:", e)


# -------------------- Tuned XGBoost Classification Report and AUC-ROC --------------------
print("\nClassification Report (Tuned XGBoost - Test Data):")
print(classification_report(y_test, xgb_test_preds, target_names=[str(cls) for cls in label_encoder.classes_]))

try:
    tuned_probs = xgb_best_model.predict_proba(X_test)
    auc_tuned = roc_auc_score(tf.keras.utils.to_categorical(y_test), tuned_probs, multi_class='ovr', average='macro')
    print("AUC-ROC (macro average OvR) Tuned XGBoost:", auc_tuned)
except Exception as e:
    print("AUC-ROC (Tuned XGBoost) could not be calculated:", e)
